In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q datasets
!pip install -q transformers

     |████████████████████████████████| 163kB 8.0MB/s 
     |████████████████████████████████| 245kB 27.6MB/s 
     |████████████████████████████████| 17.7MB 205kB/s 
     |████████████████████████████████| 1.4MB 9.0MB/s 
     |████████████████████████████████| 890kB 13.6MB/s 
     |████████████████████████████████| 2.9MB 41.1MB/s 


In [4]:
import os
import pathlib
import pandas as pd

CACHE_DIR = "./drive/Shared drives/CS 269: Recipe/tmp"
pathlib.Path(CACHE_DIR).mkdir(exist_ok=True)
dataset_path = os.path.join(CACHE_DIR, 'recipes.pkl')

if not os.path.exists(dataset_path):
    raise SystemExit("Run preprocess_pickle.ipynb to generate data file before continuing")
else:
    recipes = pd.read_pickle(dataset_path)

# TODO: Remove subsetting for final training
recipes = recipes[:20000]

def get_model_path(model_name, nickname):
    model_filename = f"{os.path.basename(model_name)}_{nickname}"
    print(model_filename)
    return os.path.join(CACHE_DIR, model_filename)

# Text generation function

In [5]:
# Download generation script
!curl -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11046  100 11046    0     0  35747      0 --:--:-- --:--:-- --:--:-- 35747


In [6]:
# Parameters for text generation
GENERATE_PARAMS = [
    "temperature",  # Temperature of 1.0 has no effect, lower tends toward greedy sampling
    "length",       # Length of generated text
    "prefix",       # Text added prior to input
    "stop_token"    # Token at which text generation is stopped
]

# Generation function
def generate(model_name, model_path, prompt, **kwargs):
    # Parse text generation parameters
    params = [f"--{arg}='{value}'" for (arg, value) in kwargs.items() if arg in GENERATE_PARAMS]
    param_str = ' '.join(params)
    print(params)

    # Generate text based on prompt
    !python run_generation.py \
        --model_type="{model_name}" \
        --model_name_or_path="{model_path}" \
        --prompt="{prompt}" \
        {param_str}

def recipe_to_str(recipe):
    # Combine components of recipe into a string
    return f"<BOS>{recipe.title}<ING>{recipe.ingredients}<INS>{recipe.instructions}<EOS>"

recipe_strings = recipes.apply(recipe_to_str, axis=1)

## Generate recipes with our best hyperparameters

In [7]:
def generate_recipe(title, ingredients=[], generate_more_ingredients=True):
    ingredients = '\n'.join(map(lambda i: f"• {i}", ingredients))
    prompt = f"{title}<ING>{ingredients}"

    if not generate_more_ingredients:
        prompt += "\n<INS>"

    model_path = get_model_path('gpt2', 'fp16_recipe_20000_lr_5e-5_wd_1e-1')
    generate('gpt2', model_path, prompt, temperature=0.7, length=400, prefix='<BOS>', stop_token='<EOS>')

In [20]:
generate_recipe("Fried Rice", ['Rice', 'egg', 'green onion'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:49:45.155252: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:49:47 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:49:57 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Fried Rice<ING>• Rice\n• egg\n• green onion', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Fried Rice<ING>• Rice
• egg
• green onion, chopped• 1/2 teaspoon salt• 1/4 t

In [18]:
generate_recipe("banh mi", ['cilantro', 'daikon radish', 'carrots', 'cucumbers'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:47:33.182316: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:47:35 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:47:45 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='banh mi<ING>• cilantro\n• daikon radish\n• carrots\n• cucumbers', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
banh mi<ING>• cilantro
• daikon radish
• carrots
• cucum

In [14]:
generate_recipe("Celery Casserole")

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:31:28.133528: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:31:30 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:31:40 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Celery Casserole<ING>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Celery Casserole<ING>• 1 (10.75 ounce) can condensed cream of celery soup• 1 cup sour cream• 1/4 c

In [15]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:35:59.875113: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:36:01 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:36:11 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
• ketchu

In [16]:
generate_recipe("Texas Pecan and Chocolate Pie", ['chocolate'])

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:36:22.698996: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:36:24 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:36:34 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Texas Pecan and Chocolate Pie<ING>• chocolate', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Texas Pecan and Chocolate Pie<ING>• chocolate-covered pretzels• 1 (9 inch)

In [17]:
generate_recipe("Casserole", ['pasta', 'tuna', 'cheese', 'jalapeno', 'ketchup'], False)

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:38:41.555192: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:38:43 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:38:53 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Casserole<ING>• pasta\n• tuna\n• cheese\n• jalapeno\n• ketchup\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Casserole<ING>• pasta
• tuna
• cheese
• jalapeno
•

In [13]:
generate_recipe("Mushroom Tacos", ['tortilla', 'mushroom', 'cheese', 'lettuce'], False)

gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1
["--temperature='0.7'", "--length='400'", "--prefix='<BOS>'", "--stop_token='<EOS>'"]
2020-12-03 00:28:44.762492: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/03/2020 00:28:46 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
12/03/2020 00:28:56 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=400, model_name_or_path='./drive/Shared drives/CS 269: Recipe/tmp/gpt2_fp16_recipe_20000_lr_5e-5_wd_1e-1', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='<BOS>', prompt='Mushroom Tacos<ING>• tortilla\n• mushroom\n• cheese\n• lettuce\n<INS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=0.7, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Mushroom Tacos<ING>• tortilla
• mushroom
• cheese
